In [1]:
import pandas as pd
import sklearn

import service
from classification.OneRClassifier import OneRClassifier
from classification.NaiveBayesClassifier import NaiveBayesClassifier
from classification.DecisionTreeClassifier import DecisionTreeClassifier
from classification.KNeighborsClassifier import KNeighborsClassifier

### data_4x

In [2]:
df = pd.read_csv('data/data_4x.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   x0      10 non-null     int64
 1   x1      10 non-null     int64
 2   x2      10 non-null     int64
 3   x3      10 non-null     int64
 4   y       10 non-null     int64
dtypes: int64(5)
memory usage: 528.0 bytes


In [3]:
X, y = df.drop(columns='y'), df['y']
X.shape, y.shape

((10, 4), (10,))

In [4]:
y.replace(0, 'y0', inplace=True)
y.replace(1, 'y1', inplace=True)
y = list(y)

In [5]:
X_pred = pd.DataFrame({'x0': [2], 'x1': [1], 'x2': [1], 'x3': [1]})

#### OneRClassifier

In [6]:
clf = OneRClassifier()
clf.fit(X, y);

In [7]:
service.OneRClassifier_info(clf)

x[3] == 0: y = y1
x[3] == 1: y = y0


In [8]:
clf.predict(X_pred)

array(['y0'], dtype='<U2')

#### NaiveBayesClassifier

In [9]:
clf = NaiveBayesClassifier()
clf.fit(X, y);

In [10]:
service.NaiveBayesClassifier_info(clf)

class_log_probs_:
-0.916290731874155 -0.5108256237659907 

feature_log_probs_:
y  feat_v
0  0        -0.559616
   1        -1.252763
1  2        -0.810930
   0        -1.098612
   1        -1.504077
Name: count, dtype: float64 y  feat_v
0  0        -0.693147
   1        -0.693147
1  0        -0.693147
   1        -0.693147
Name: count, dtype: float64 y  feat_v
0  1        -0.405465
   0        -1.098612
1  0        -0.693147
   1        -0.693147
Name: count, dtype: float64 y  feat_v
0  1        -0.405465
   0        -1.098612
1  0        -0.287682
   1        -1.386294
Name: count, dtype: float64 

feature_missing_log_probs_:
y
0   -1.945910
1   -2.197225
Name: feat_v, dtype: float64 y
0   -1.791759
1   -2.079442
Name: feat_v, dtype: float64 y
0   -1.791759
1   -2.079442
Name: feat_v, dtype: float64 y
0   -1.791759
1   -2.079442
Name: feat_v, dtype: float64 



In [11]:
clf.predict(X_pred)

array(['y1'], dtype='<U2')

Compare with similar scikit-learn model

In [12]:
sk_clf = sklearn.naive_bayes.CategoricalNB()
sk_clf.fit(X, y);

In [13]:
service.sklearn_CategoricalNB_info(sk_clf)

class_log_prior_:
-0.9162907318741553 -0.5108256237659909 

feature_log_prob_:
[[-0.55961579 -1.25276297 -1.94591015]
 [-1.09861229 -1.5040774  -0.81093022]] [[-0.69314718 -0.69314718]
 [-0.69314718 -0.69314718]] [[-1.09861229 -0.40546511]
 [-0.69314718 -0.69314718]] [[-1.09861229 -0.40546511]
 [-0.28768207 -1.38629436]] 



In [14]:
sk_clf.predict(X_pred)

array(['y1'], dtype='<U2')

#### DecisionTreeClassifier

In [15]:
clf = DecisionTreeClassifier()
clf.fit(X, y);

In [16]:
service.DecisionTreeClassifier_info(clf)

└── Feature[1]
    (0)
    ├── Feature[4]
    │   (0)
    │   ├── Feature[2]
    │   │   (0)
    │   │   ├── Label: 0
    │   │   (1)
    │   │   └── Label: 1
    │   (1)
    │   └── Label: 0
    (1)
    ├── Feature[2]
    │   (0)
    │   ├── Feature[3]
    │   │   (0)
    │   │   ├── Feature[4]
    │   │   │   (1)
    │   │   │   ├── Label: 1
    │   │   │   (0)
    │   │   │   └── Label: 1
    │   │   (1)
    │   │   └── Label: 1
    │   (1)
    │   └── Label: 1
    (2)
    └── Label: 1


In [17]:
clf.predict(X_pred)

array(['y1'], dtype='<U2')

#### KNeighborsClassifier

In [18]:
n_neighbors = 3

##### weights='uniform'

In [19]:
clf = KNeighborsClassifier(n_neighbors=n_neighbors, weights='uniform')
clf.fit(X, y);

In [20]:
service.KNeighborsClassifier_info(clf, X_pred)

neighbors_dist:
[1.         1.41421356 1.73205081] 

neighbors_indices:
[8 2 1] 



In [21]:
clf.predict(X_pred)

array(['y1'], dtype='<U2')

##### weights='distance'

In [22]:
clf = KNeighborsClassifier(n_neighbors=n_neighbors, weights='distance')
clf.fit(X, y);

In [23]:
clf.predict(X_pred)

array(['y1'], dtype='<U2')

Compare with similar scikit-learn model

In [24]:
sk_clf = sklearn.neighbors.KNeighborsClassifier(n_neighbors=n_neighbors)
sk_clf.fit(X, y);

In [25]:
service.sklearn_KNeighborsClassifier_info(sk_clf, X_pred)

neighbors_dist:
[1.         1.41421356 1.73205081] 

neighbors_indices:
[8 2 1] 



In [26]:
sk_clf.predict(X_pred)

array(['y1'], dtype='<U2')

##### weights='distance_squared'

In [27]:
clf = KNeighborsClassifier(n_neighbors=n_neighbors, weights='distance_squared')
clf.fit(X, y);

In [28]:
clf.predict(X_pred)

array(['y1'], dtype='<U2')

### More datasets (data_3x)

In [29]:
classifiers = [
    OneRClassifier(),
    NaiveBayesClassifier(),
    DecisionTreeClassifier(),
    KNeighborsClassifier(weights='uniform'),
    KNeighborsClassifier(weights='distance'),
    KNeighborsClassifier(weights='distance_squared'),
]
sk_classifiers = [
    sklearn.naive_bayes.CategoricalNB(),
    sklearn.tree.DecisionTreeClassifier(criterion='entropy'),
    sklearn.neighbors.KNeighborsClassifier(weights='uniform'),
    sklearn.neighbors.KNeighborsClassifier(weights='distance'),

    sklearn.dummy.DummyClassifier(),
]

In [30]:
df = pd.read_csv('data/data_3x.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   dataset  40 non-null     int64
 1   x0       40 non-null     int64
 2   x1       40 non-null     int64
 3   x2       40 non-null     int64
 4   y        40 non-null     int64
dtypes: int64(5)
memory usage: 1.7 KB


In [31]:
res_df = pd.DataFrame()
X_pred = pd.DataFrame({'x0': [1], 'x1': [1], 'x2': [1]})
for clf in classifiers:
    for df_i, df_v in df.groupby('dataset'):
        df_v = df_v.drop(columns=['dataset'])
        X, y = df_v.drop(columns='y'), df_v['y']
        clf.fit(X, y)
        y_pred = clf.predict(X_pred)[0]
        res_df.loc[str(clf), df_i] = y_pred

res_df.astype(int)

,1,2,3,4
OneRClassifier(),1,1,1,1
NaiveBayesClassifier(),0,1,0,1
DecisionTreeClassifier(),0,1,0,1
KNeighborsClassifier(),1,1,0,1
KNeighborsClassifier(weights='distance'),1,1,0,1
KNeighborsClassifier(weights='distance_squared'),1,1,0,1


### Student performance

In [32]:
df = pd.read_csv('data/student_performance.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2392 entries, 0 to 2391
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   StudentID          2392 non-null   int64  
 1   Age                2392 non-null   int64  
 2   Gender             2392 non-null   int64  
 3   Ethnicity          2392 non-null   int64  
 4   ParentalEducation  2392 non-null   int64  
 5   StudyTimeWeekly    2392 non-null   float64
 6   Absences           2392 non-null   int64  
 7   Tutoring           2392 non-null   int64  
 8   ParentalSupport    2392 non-null   int64  
 9   Extracurricular    2392 non-null   int64  
 10  Sports             2392 non-null   int64  
 11  Music              2392 non-null   int64  
 12  Volunteering       2392 non-null   int64  
 13  GPA                2392 non-null   float64
 14  GradeClass         2392 non-null   float64
dtypes: float64(3), int64(12)
memory usage: 280.4 KB


In [33]:
X = df[[
    'Age', 'Gender', 'Ethnicity', 'ParentalEducation', 'Absences', 'Tutoring',
    'ParentalSupport', 'Extracurricular', 'Sports', 'Music', 'Volunteering'
]]
y = df['GradeClass']
X.shape, y.shape

((2392, 11), (2392,))

In [34]:
from sklearn.model_selection import train_test_split

test_size_ratio = 0.2
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=test_size_ratio,
    stratify=y,
    random_state=42
)

print(f"Train: X={X_train.shape} y={y_train.shape}")
print(f"Test: X={X_test.shape} y={y_test.shape}")

Train: X=(1913, 11) y=(1913,)
Test: X=(479, 11) y=(479,)


In [35]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder()
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

In [36]:
from sklearn.metrics import accuracy_score

res_df = pd.DataFrame()
for i, clf in enumerate(classifiers + sk_classifiers):
    res_df.loc[i, 'clf'] = str(clf)
    res_df.loc[i, 'impl'] = 'custom' if i < len(classifiers) else 'sklearn'
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    res_df.loc[i, 'accuracy'] = accuracy
res_df.sort_values(by=['accuracy'], ascending=False, inplace=True)

res_df

,clf,impl,accuracy
1,NaiveBayesClassifier(),custom,0.680585
6,CategoricalNB(),sklearn,0.680585
3,KNeighborsClassifier(),custom,0.665971
4,KNeighborsClassifier(weights='distance'),custom,0.661795
8,KNeighborsClassifier(),sklearn,0.661795
5,KNeighborsClassifier(weights='distance_squared'),custom,0.659708
9,KNeighborsClassifier(weights='distance'),sklearn,0.659708
0,OneRClassifier(),custom,0.640919
7,DecisionTreeClassifier(criterion='entropy'),sklearn,0.578288
2,DecisionTreeClassifier(),custom,0.576200


### Check classifiers (scikit-learn)

In [37]:
classifiers = [
    OneRClassifier(),
    NaiveBayesClassifier(),
    DecisionTreeClassifier(),
    KNeighborsClassifier(),
]

In [38]:
from sklearn.utils.estimator_checks import estimator_checks_generator

total_checks = 0
skipped_checks = 0
for clf in classifiers:
    for (estimator, check) in estimator_checks_generator(clf):
        total_checks += 1
        try:
            check(estimator)
        except KeyError as e:
            if "encountered an unknown value" in str(e):
                skipped_checks += 1
                continue
            raise

print("Complete.")
print(f"Total checks: {total_checks}")
print(f"Skipped checks: {skipped_checks} ({skipped_checks/total_checks*100:.2f}%)")

KeyError: np.float64(98.77456448116983)